# Hyperparameter tuning
This notebook explores hyperparameter tuning. It uses the boston house price dataset built into Sklearn.

## Import

In [1]:
# Core libraries
import pandas as pd

# Sklearn processing
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

# Sklearn regression algorithms
from sklearn.neighbors import KNeighborsRegressor

# Sklearn regression model evaluation functions
from sklearn.metrics import r2_score

## Load data, split into X and y and scale data

In [2]:
# Load Boston housing data set
boston = pd.read_csv("boston.csv")

# Define the X (input) and y (target) features
X = boston.drop("MEDV", axis=1)
y = boston["MEDV"]

# Rescale the input features
scaler = MinMaxScaler(feature_range=(0,1))
X = scaler.fit_transform(X)

## Build a model with default hyperparameters

In [3]:
# Create an empty model
model = KNeighborsRegressor()

In [4]:
# Inspect the model's default hyperparameters:
model

KNeighborsRegressor()

In [5]:
# What hyperparameters can we tune?
model.get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 5,
 'p': 2,
 'weights': 'uniform'}

## Tune hyperparameters with grid search 

In [6]:
# Select an algorithm
algorithm = KNeighborsRegressor()

# Create 3 folds
seed = 13
kfold = KFold(n_splits=3, shuffle=True, random_state=seed)

# Define our candidate hyperparameters
hp_candidates = [{'n_neighbors': [2,3,4,5,6], 'weights': ['uniform','distance']}]

# Search for best hyperparameters
grid = GridSearchCV(estimator=algorithm, param_grid=hp_candidates, cv=kfold, scoring='r2')
grid.fit(X, y)

# Get the results
print(grid.best_score_)
print(grid.best_estimator_)
print(grid.best_params_)

0.7664342242670098
KNeighborsRegressor(n_neighbors=3, weights='distance')
{'n_neighbors': 3, 'weights': 'distance'}


### Get a full breakdown of the grid search

In [7]:
grid.cv_results_

{'mean_fit_time': array([0.00366489, 0.0019908 , 0.00133332, 0.00133204, 0.00166535,
        0.00266425, 0.00200129, 0.0033323 , 0.00166599, 0.00133316]),
 'std_fit_time': array([4.73000122e-04, 8.15444962e-04, 4.72157255e-04, 4.70527427e-04,
        4.70696366e-04, 4.72494310e-04, 2.36022352e-06, 1.24823335e-03,
        4.71482745e-04, 4.71089385e-04]),
 'mean_score_time': array([0.00366489, 0.0033466 , 0.00298985, 0.00333158, 0.00299994,
        0.00466426, 0.00532818, 0.00366561, 0.00299756, 0.0026648 ]),
 'std_score_time': array([1.24704535e-03, 4.58276991e-04, 8.17162799e-04, 9.43358987e-04,
        2.65015475e-06, 4.71426640e-04, 4.73394403e-04, 1.24717123e-03,
        5.94720425e-07, 4.70696044e-04]),
 'param_n_neighbors': masked_array(data=[2, 2, 3, 3, 4, 4, 5, 5, 6, 6],
              mask=[False, False, False, False, False, False, False, False,
                    False, False],
        fill_value='?',
             dtype=object),
 'param_weights': masked_array(data=['uniform',

### Prove that best_score_ is the mean of all the k-fold scores
Here's a little check to see how best_score_ is derived from cv_results_

In [8]:
# Get the index of the best hyperparameter combination chosen by GridSearchCv()
grid.best_index_

3

In [9]:
# Get the mean and std of the k-fold scores for the best hyperparameter combination
print(grid.cv_results_['mean_test_score'][grid.best_index_])
print(grid.cv_results_['std_test_score'][grid.best_index_])

0.7664342242670098
0.018054321344618637
